In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np
from time import time

# Program bitstream to FPGA
overlay = Overlay('/home/xilinx/design_1.bit')

# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = overlay.axi_dma_0.sendchannel
dma_recv = overlay.axi_dma_0.recvchannel

In [2]:
# Allocate physical memory
input_buffer = allocate(shape=(44,), dtype=np.uint32)
output_buffer = allocate(shape=(8,), dtype=np.uint32)

In [3]:
# Measure the time required for HW ANN
t1 = time()
# Weight and bias
input_buffer[0] = 0x0533fb33
input_buffer[1] = 0xfacc06cc
input_buffer[2] = 0x0000facc
input_buffer[3] = 0x00000000
input_buffer[4] = 0x02000133
input_buffer[5] = 0x040000cc
input_buffer[6] = 0x0000fc00
input_buffer[7] = 0x00000000
input_buffer[8] = 0x00660266
input_buffer[9] = 0x06000333
input_buffer[10] = 0x0000fc00
input_buffer[11] = 0x00000000
input_buffer[12] = 0xfb330533
input_buffer[13] = 0x0533fa66
input_buffer[14] = 0x0000fc66
input_buffer[15] = 0x00000000
input_buffer[16] = 0x01330533
input_buffer[17] = 0x01990200
input_buffer[18] = 0x0000fc00
input_buffer[19] = 0x00000000
input_buffer[20] = 0xfecc14cc
input_buffer[21] = 0xf2000333
input_buffer[22] = 0xfa000066
input_buffer[23] = 0x00000000
input_buffer[24] = 0x0066eccc
input_buffer[25] = 0x100002cc
input_buffer[26] = 0xfa660399
input_buffer[27] = 0x00000000
# Input
input_buffer[28] = 0x1c000800
input_buffer[29] = 0x0c001800
input_buffer[30] = 0x0c001800
input_buffer[31] = 0x00000000
input_buffer[32] = 0x08002800
input_buffer[33] = 0x28002000
input_buffer[34] = 0x08002400
input_buffer[35] = 0x00000000
input_buffer[36] = 0x0c001400
input_buffer[37] = 0x0c000400
input_buffer[38] = 0x18001400
input_buffer[39] = 0x00000000
input_buffer[40] = 0x0c000c00
input_buffer[41] = 0x04001800
input_buffer[42] = 0x28001800
input_buffer[43] = 0x00000000
for i in range(1000000):
    # Do AXI DMA MM2S transfer
    dma_send.transfer(input_buffer)
    # Do AXI DMA S2MM transfer
    dma_recv.transfer(output_buffer)
# Output
pred_hw = np.zeros((2,6))
pred_hw[0][0] = ((output_buffer[0] & 0x0000FFFF) / 1024.0)
pred_hw[0][1] = (((output_buffer[0] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[0][2] = ((output_buffer[1] & 0x0000FFFF) / 1024.0)
pred_hw[0][3] = (((output_buffer[1] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[0][4] = ((output_buffer[2] & 0x0000FFFF) / 1024.0)
pred_hw[0][5] = (((output_buffer[2] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[1][0] = ((output_buffer[4] & 0x0000FFFF) / 1024.0)
pred_hw[1][1] = (((output_buffer[4] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[1][2] = ((output_buffer[5] & 0x0000FFFF) / 1024.0)
pred_hw[1][3] = (((output_buffer[5] & 0xFFFF0000) >> 16) / 1024.0)
pred_hw[1][4] = ((output_buffer[6] & 0x0000FFFF) / 1024.0)
pred_hw[1][5] = (((output_buffer[6] & 0xFFFF0000) >> 16) / 1024.0)
t2 = time()
t_diff = t2 - t1
print('Time used for HW ANN: {}s'.format(t_diff))

Time used for HW ANN: 424.38378167152405s


In [4]:
# Check prediction
np.round(pred_hw)

array([[1., 0., 0., 1., 1., 0.],
       [0., 1., 1., 0., 0., 1.]])

In [5]:
# Measure the time required for SW ANN
t1 = time()
k = np.array([[ 2, 7, 6,  3, 6,  3],
              [10, 2, 8, 10, 9,  2],
              [ 5, 3, 1,  3, 5,  6],
              [ 3, 3, 6,  1, 6, 10]])
wb2 = np.array([[-1.2,  1.3,  1.7, -1.3, -1.3],
                [ 0.3,  0.5,  0.2,  1.0, -1.0],
                [ 0.6,  0.1,  0.8,  1.5, -1.0],
                [ 1.3, -1.2, -1.4,  1.3, -0.9],
                [ 1.3,  0.3,  0.5,  0.4, -1.0]])
wb3 = np.array([[ 5.2, -0.3, 0.8, -3.5, 0.1, -1.5],
                [-4.8,  0.1, 0.7,  4.0, 0.9, -1.4]])
for i in range(1000000):
    k_padded = np.concatenate((k, np.array([[1, 1, 1, 1, 1, 1]])), axis=0)
    z2 = np.matmul(wb2, k_padded)
    a2 = 1/(1+np.exp(-z2))
    a2_padded = np.concatenate((a2, np.array([[1, 1, 1, 1, 1, 1]])), axis=0)
    z3 = np.matmul(wb3, a2_padded)
    a3 = 1/(1+np.exp(-z3))
t2 = time()
t_diff = t2 - t1
print('Time used for SW ANN: {}s'.format(t_diff))

Time used for SW ANN: 504.0776844024658s


In [6]:
# Check prediction
np.round(a3)

array([[1., 0., 0., 1., 1., 0.],
       [0., 1., 1., 0., 0., 1.]])